This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [65]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

In [27]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

url = "https://data.nasdaq.com/api/v3/datasets/WIKI/FB/data.json?api_key="+API_KEY

r = requests.get(url)

data = r.json()

print(data['dataset_data'].keys())
print()
print(data['dataset_data']['column_names'])
print()
for row in data['dataset_data']['data'][:1]:
    print(row)
    print()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']

['2018-03-27', 156.31, 162.85, 150.75, 152.19, 76787884.0, 0.0, 1.0, 156.31, 162.85, 150.75, 152.19, 76787884.0]



These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [42]:
#Url to access AFX_X data for the year 2017
url_AFX= 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY

#Make the request to the API
request = requests.get(url_AFX)

AFX_data = request.json()


#Creating a new dictionary that takes the json response, gets all of the column names
#assigns them as keys then loop through the actual data and assignes all of the data values
#to the appropriate keys. 
data_dict = {col: [] for col in AFX_data['dataset_data']['column_names']}

#Loop through data and assign values to keys to make searching easier. 
for row in AFX_data['dataset_data']['data']:
    for col, value in zip(AFX_data['dataset_data']['column_names'], row):
        data_dict[col].append(value)

In [44]:
print(data_dict.keys())

dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])


In [48]:
#Finding the Highest and Lowest prices for this period

#Start by creating variables that will store the highest and lowest price along with the 
#indicies of those values (in order to get all of the data for those prices)
highest_price = None
lowest_price = None
highest_price_index = None
lowest_price_index = None

#Iterate through the prices in the 'Open' list and compare them to find the highest and 
#lowest. 

for index, price in enumerate(data_dict['Open']):
    if price is not None:
        if highest_price is None or price > highest_price:
            highest_price = price
            highest_price_index = index
        if lowest_price is None or price < lowest_price:
            lowest_price = price
            lowest_price_index = index
print(f"Highest Opening Price: {highest_price}, Index: {highest_price_index}")
print(f"Lowest Opening Price: {lowest_price}, Index: {lowest_price_index}")

Highest Opening Price: 53.11, Index: 9
Lowest Opening Price: 34.0, Index: 238


In [50]:
#This function will display all of the information for a given index and dictionary
#I created this in order to not only get the highest and lowest price but to also
#get all of the information for those days as well. 
#This function was extermemly helpful for this assignment O_o

def data_for_index(data_dict, index):
    for col in data_dict:
        print(f"{col}: {data_dict[col][index]}")
        
print("\nHighest Opening Price Data:")
data_for_index(data_dict, highest_price_index)

print("\nLowest Opening Price Data:")
data_for_index(data_dict, lowest_price_index)


Highest Opening Price Data:
Date: 2017-12-14
Open: 53.11
High: 53.54
Low: 52.15
Close: 52.67
Change: None
Traded Volume: 132981.0
Turnover: 7016953.0
Last Price of the Day: None
Daily Traded Units: None
Daily Turnover: None

Lowest Opening Price Data:
Date: 2017-01-24
Open: 34.0
High: 34.35
Low: 33.85
Close: 34.22
Change: None
Traded Volume: 48797.0
Turnover: 1666086.0
Last Price of the Day: None
Daily Traded Units: None
Daily Turnover: None


In [55]:
#Finding the largest change in one day based on the difference in open and close price

#Start by creating variables that will keep track of the index and chage each day
largest_change = 0
largest_change_index = None

#Loop through all of the dates and pull out the high and low values to create the difference
#and compare
for index in range(len(data_dict['Date'])):
    high = data_dict['High'][index]
    low = data_dict['Low'][index]
    
    #Just making sure the values exist
    if high is not None and low is not None:
        #Create the difference to compare
        change = high - low
        
        #Compare =]
        if change > largest_change:
            largest_change = change
            largest_change_index = index

print(f"The largest change is: {largest_change}")
print("\nLargest Change Price Data:")
data_for_index(data_dict, highest_price_index)

The largest change is: 2.8100000000000023

Largest Change Price Data:
Date: 2017-12-14
Open: 53.11
High: 53.54
Low: 52.15
Close: 52.67
Change: None
Traded Volume: 132981.0
Turnover: 7016953.0
Last Price of the Day: None
Daily Traded Units: None
Daily Turnover: None


In [60]:
#Finding the largest change between any two consecutive days based on Closing Price

largest_closing_change = 0
closing_change_index = None

#I dont want to get to the last index as there will be nothing to compre it to.
for i in range(len(data_dict['Date']) - 1):
    closing = data_dict['Close'][i]
    closing_next_day = data_dict['Close'][i + 1]

    #Use abs value here to compare as a larger negative change could be bigger than a 
    #small positive change
    if closing is not None and closing_next_day is not None:
        change = abs(closing_next_day - closing)

        if change > largest_closing_change:
            largest_closing_change = change
            closing_change_index = i


print(f"Largest Closing Price Change: {largest_closing_change}")
print(f"Occurred between the dates: {data_dict['Date'][closing_change_index]} and {data_dict['Date'][closing_change_index + 1]}")
print()
data_for_index(data_dict, closing_change_index)
print()
data_for_index(data_dict, closing_change_index+1)

Largest Closing Price Change: 2.559999999999995
Occurred between the dates: 2017-08-09 and 2017-08-08

Date: 2017-08-09
Open: 43.5
High: 43.5
Low: 41.64
Close: 41.81
Change: None
Traded Volume: 355857.0
Turnover: 15003956.0
Last Price of the Day: None
Daily Traded Units: None
Daily Turnover: None

Date: 2017-08-08
Open: 44.9
High: 45.09
Low: 44.15
Close: 44.37
Change: None
Traded Volume: 156168.0
Turnover: 6941408.0
Last Price of the Day: None
Daily Traded Units: None
Daily Turnover: None


In [61]:
#Find the average daily trading volume during this year

# Create a list of all traded volume values
traded_volumes = [volume for volume in data_dict['Traded Volume'] if volume is not None]

# Calculate the sum the traded_volumes list
total_volume = sum(traded_volumes)

# Calculate the average by dividing the total volume by the number of trading days
average_volume = total_volume / len(traded_volumes)

print(f"Average Trading Volume for the Year: {average_volume}")


Average Trading Volume for the Year: 89124.33725490196


In [62]:
#Finding the median trading function for the year (As a function, although now I wish i created all of the calculations as function)

def find_median_trading_volume(data_dict):
    #Create the list of traded valume and sort it
    traded_volumes = [volume for volume in data_dict['Traded Volume'] if volume is not None]
    sorted_volumes = sorted(traded_volumes)

    #Calculate the median
    list_length = len(sorted_volumes)
    middle = list_length // 2
    
    #If even number of elements
    if list_length % 2 == 0:
        median_volume = (sorted_volumes[middle - 1] + sorted_volumes[middle]) / 2
        
    #If odd number of elements 
    else:  
        median_volume = sorted_volumes[middle]

    #Find the index of the median volume in the original list
    median_index = traded_volumes.index(median_volume)

    #Return the median volume and its index
    return median_volume, median_index



In [64]:
# Call the function
median_volume, median_index = find_median_trading_volume(data_dict)

# Print the results
print(f"Median Trading Volume for the Year: {median_volume}")
print("\nData for the Day with the Median Trading Volume:")
data_for_index(data_dict, median_index)

#Adding a comment here to say that this function only worked becasue the length of the traded
#values list was odd, if the list was even then median_index would not be just one number
#I would need to account for this ineither the median function or the data_for_index function

Median Trading Volume for the Year: 76286.0

Data for the Day with the Median Trading Volume:
Date: 2017-11-13
Open: 48.38
High: 48.61
Low: 47.76
Close: 48.34
Change: None
Traded Volume: 76286.0
Turnover: 3681337.0
Last Price of the Day: None
Daily Traded Units: None
Daily Turnover: None
